# Pandas for Exploratory Data Analysis II 

Recall Pandas is the most useful Python library for data manipulation and exploration. We have so much more to see!

In this lesson, we'll continue exploring Pandas for EDA. Specifically: 

- Identify and handle missing values with Pandas.
- Implement groupby statements for specific segmented analysis.
- Use apply functions to clean data with Pandas.

We'll implicitly review many functions from our first Pandas lesson along the way!

## Remember the Iowa Liquor Dataset?

- **Invoice/Item Number** - Concatenated invoice and line number associated with the liquor order. This provides a unique identifier for the individual liquor products included in the store order
- **Date** - Date of order 
- **Store Number** - Unique number assigned to the store who ordered the liquor.
- **Store Name** - Name of store who ordered the liquor.
- **Address** - Address of the store that ordered the liquor
- **City** - City where the store who ordered the liquor is located
- **Zip Code** - Zip Code of where the store that ordered is located 
- **Store Location** - Location of store who ordered the liquor. The Address, City, State and Zip Code are geocoded to provide geographic coordinates. Accuracy of geocoding is dependent on how well the address is interpreted and the completeness of the reference data used.
- **County Number** - Iowa county number for the county where store who ordered the liquor is located
- **County** - County where the store who ordered the liquor is located
- **Category** - Category code associated with the liquor ordered
- **Category Names** - Category of the liquor ordered.
- **Vendor Number** - The vendor number of the company for the brand of liquor ordered
- **Vendor Name** - The vendor name of the company for the brand of liquor ordered
- **Item Name** - Item number for the individual liquor product ordered.
- **Item Description** - Description of the individual liquor product ordered.
- **Pack** - The number of bottles in a case for the liquor ordered
- **Bottle Volume (mL)** - Volume of each liquor bottle ordered in milliliters.
- **State Bottle Cost** - The amount that Alcoholic Beverages Division paid for each bottle of liquor ordered
- **State Bottle Retail** - The amount the store paid for each bottle of liquor ordered
- **Bottles Solde** - The number of bottles of liquor ordered by the store
- **Sale (Dollars)** - Total cost of liquor order (number of bottles multiplied by the state bottle retail)
- **Volume Sold (Liters)** - Total volume of liquor ordered in liters. (i.e. (Bottle Volume (ml) x Bottles Sold)/1,000)
- **Volume Sold (Gallons)** - Total volume of liquor ordered in gallons. (i.e. (Bottle Volume (ml) x Bottles Sold)/3785.411784)


### Our Modified Iowa Liquor Dataset

Because the full dataset (of all liquor sales from 2012 to present) is greater than 13 million rows (13,948,103+ at the time of writing), **we will work with a modified dataset.**

Our modified dataset has a few key changes:
- Only sales from May 2017 and May 2018 are present
- A number of values have been deliberately deleted (to practice working with missing data!)


### Import Pandas

In [1]:
import pandas as pd
import numpy as np # used for linear algebra and random sampling
%matplotlib inline

### Read in the dataset

We are using the `read_csv()` method (and using a special encoding to handle our file's Excel roots).

In [2]:
liq = pd.read_csv("../data/iowa_liquor_may_17_18.csv", encoding='cp1252')

In [3]:
# remember checking the top five rows
liq.head()

,Date,Store Number,Store Name,City,Zip Code,Store Location,County,Category Name,Vendor Name,Item Number,...,Pack,Bottle Volume (ml),State Bottle Cost,State Bottle Retail,Bottles Sold,Sale (Dollars),Volume Sold (Liters),Volume Sold (Gallons),is_may_2017,is_may_2018
0,5/2/17,5286,Sauce,Iowa City,52240.0,"108, College\rIowa City 52240\r",JOHNSON,Blended Whiskies,Laird & Company,23827,...,12,"1,000",$4.40,$6.60,1,$79.20,1.00,0.26,1,0
1,5/1/17,4169,Super Quick 2 / Hubbell,Des Moines,50317.0,1824 Hubbell Ave\rDes Moines 50317\r,POLK,Canadian Whiskies,CONSTELLATION BRANDS INC,11773,...,48,200,$1.56,$2.34,1,$112.32,0.20,0.05,1,0
2,5/1/17,2641,Hy-Vee Drugstore / Council Bluffs,Council Bluffs,51501.0,757 W Broadway\rCouncil Bluffs 51501\r(41.2616...,POTTAWATTA,American Cordials & Liqueur,SAZERAC NORTH AMERICA,84207,...,10,600,$6.00,$9.00,1,$9.00,0.60,0.15,1,0
3,5/1/17,2641,Hy-Vee Drugstore / Council Bluffs,Council Bluffs,51501.0,757 W Broadway\rCouncil Bluffs 51501\r(41.2616...,POTTAWATTA,American Cordials & Liqueur,SAZERAC NORTH AMERICA,84197,...,10,600,$6.00,$9.00,1,$9.00,0.60,0.15,1,0
4,5/3/17,2565,Hy-Vee Food Store / Spencer,Spencer,51301.0,"819 N Grand Ave\rSpencer 51301\r(43.145897, -9...",CLAY,Mixto Tequila,LUXCO INC,89448,...,6,"1,750",$12.00,$18.00,3,$18.00,5.25,1.38,1,0


In [4]:
liq.shape

(427923, 21)

### Rename our columns (like last time)

Let's rename our columns so our data is easier to work with.

In [5]:
# declare a list of strings - these strings will become the new column names
cols = ['date', 'store_number', 'store_name', 'city', 
        'zip_code', 'location', 'county', 'category_name',
        'vendor_name', 'item_number', 'item_description', 'pack', 
       'bottle_vol_ml', 'state_bottle_cost', 'state_bottle_retail', 'bottles_sold',
       'sale', 'volumne_sold_l', 'volume_sold_gal', 'is_may_2017', 'is_may_2018']

In [6]:
liq.columns = cols

In [7]:
liq.columns

Index(['date', 'store_number', 'store_name', 'city', 'zip_code', 'location',
       'county', 'category_name', 'vendor_name', 'item_number',
       'item_description', 'pack', 'bottle_vol_ml', 'state_bottle_cost',
       'state_bottle_retail', 'bottles_sold', 'sale', 'volumne_sold_l',
       'volume_sold_gal', 'is_may_2017', 'is_may_2018'],
      dtype='object')

## Handling missing data

Recall missing data is a systemic, challenging problem for data scientists. Imagine conducting a US election poll, but losing all female voter responses in the process!

"Handling missing data" itself is a broad topic. We'll focus on two components:

- Using Pandas to identify we have missing data
- Strategies to fill in missing data
- Filling in missing data with Pandas


***Create missing data*** 😮

> For the purposes of education... Run the below cell to *create* missing data in our DataFrame.

In [8]:
# create random places to drop data
to_drop1 = np.random.randint(1,427923,72746)
to_drop2 = np.random.randint(1,427923,29954)
np.append(to_drop2, 2) # make sure we have index number 2 to drop


# drop the data!!!
liq.iloc[to_drop1,15] = np.nan
liq.iloc[to_drop2,16] = np.nan


### Identifying missing data

Before *handling*, we must identify we're missing data at all! (In this given dataset, we have eliminated datapoints for the purposes of these exercises.)

We have a few ways to explore missing data, and they are reminiscient of our Boolean filters.

In [9]:
# True when data isn't missing
liq.notnull() 

,date,store_number,store_name,city,zip_code,location,county,category_name,vendor_name,item_number,...,pack,bottle_vol_ml,state_bottle_cost,state_bottle_retail,bottles_sold,sale,volumne_sold_l,volume_sold_gal,is_may_2017,is_may_2018
0,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
1,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
2,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
3,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
4,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
5,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,False,False,True,True,True,True
6,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
7,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,False,True,True,True,True,True
8,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True
9,True,True,True,True,True,True,True,True,True,True,...,True,True,True,True,True,True,True,True,True,True


In [10]:
# True when data is missing
liq.isnull() 

,date,store_number,store_name,city,zip_code,location,county,category_name,vendor_name,item_number,...,pack,bottle_vol_ml,state_bottle_cost,state_bottle_retail,bottles_sold,sale,volumne_sold_l,volume_sold_gal,is_may_2017,is_may_2018
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
5,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,True,False,False,False,False
6,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
7,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False
8,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
9,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


Now, we may want to see null values in aggregate. We can use `sum()` to sum down a given column

In [11]:
# see number of missing values per column
liq.isnull().sum()

date                       0
store_number               0
store_name                 0
city                      14
zip_code                  14
location                  14
county                    14
category_name            421
vendor_name                0
item_number                0
item_description           0
pack                       0
bottle_vol_ml              0
state_bottle_cost          0
state_bottle_retail        0
bottles_sold           66817
sale                   28994
volumne_sold_l             0
volume_sold_gal            0
is_may_2017                0
is_may_2018                0
dtype: int64

Look! We've found missing values!

How could this missing data be problematic for our analysis?

### Understanding missing data

Finding missing data is the easy part! Determining way to do next is more complicated.

Typically, we are most interested in knowing **why** we are missing data. Once we know what 'type of missingness' we have (the source of missing data), we can proceed effectively.

Let's first quantify how much data we are missing.

In [12]:
# use a boolean filter to only show rows where bottles_sold is missing
liq[liq.bottles_sold.isnull()]

,date,store_number,store_name,city,zip_code,location,county,category_name,vendor_name,item_number,...,pack,bottle_vol_ml,state_bottle_cost,state_bottle_retail,bottles_sold,sale,volumne_sold_l,volume_sold_gal,is_may_2017,is_may_2018
5,5/3/17,5105,Three Brothers Liquors,North Liberty,52317.0,585 HIGHWAY 965\rNorth Liberty 52317\r(41.7381...,JOHNSON,American Vodka,A V BRANDS INC,937040,...,6,750,$21.99,$32.99,NaN,NaN,1.50,0.39,1,0
7,5/1/17,4301,Sahota Food Mart,Des Moines,50320.0,"1805 SE 14th St\rDes Moines 50320\r(41.57222, ...",POLK,American Vodkas,Laird & Company,35914,...,24,375,$1.93,$2.90,NaN,$69.60,0.37,0.09,1,0
10,5/1/17,4301,Sahota Food Mart,Des Moines,50320.0,"1805 SE 14th St\rDes Moines 50320\r(41.57222, ...",POLK,Imported Brandies,Jim Beam Brands,47790,...,24,200,$4.72,$7.08,NaN,$169.92,0.40,0.10,1,0
16,5/1/17,4301,Sahota Food Mart,Des Moines,50320.0,"1805 SE 14th St\rDes Moines 50320\r(41.57222, ...",POLK,100% Agave Tequila,DIAGEO AMERICAS,89175,...,6,750,$32.99,$49.49,NaN,$49.49,2.25,0.59,1,0
30,5/1/17,4617,Lickety Liquor,Des Moines,50317.0,2501 HUBBELL AVE\rDes Moines 50317\r(41.607866...,POLK,Tennessee Whiskies,Brown Forman Corp.,86670,...,12,750,$15.57,$23.36,NaN,$23.36,1.50,0.39,1,0
39,5/1/17,4617,Lickety Liquor,Des Moines,50317.0,2501 HUBBELL AVE\rDes Moines 50317\r(41.607866...,POLK,Imported Cordials & Liqueurs,"Sidney Frank Importing Company, Inc.",65253,...,48,200,$3.51,$5.27,NaN,$252.96,0.20,0.05,1,0
40,5/1/17,4617,Lickety Liquor,Des Moines,50317.0,2501 HUBBELL AVE\rDes Moines 50317\r(41.607866...,POLK,100% Agave Tequila,Patron Spirits Company,88294,...,12,375,$14.50,$21.75,NaN,$261.00,0.37,0.09,1,0
42,5/1/17,4617,Lickety Liquor,Des Moines,50317.0,2501 HUBBELL AVE\rDes Moines 50317\r(41.607866...,POLK,Canadian Whiskies,SAZERAC NORTH AMERICA,12407,...,12,"1,000",$5.50,$8.25,NaN,$8.25,3.00,0.79,1,0
44,5/1/17,2190,"Central City Liquor, Inc.",Des Moines,50314.0,"1460 2ND AVE\rDes Moines 50314\r(41.60566, -93...",POLK,American Vodkas,McCormick Distilling Co.,36908,...,6,"1,750",$7.47,$11.21,NaN,$67.26,1.75,0.46,1,0
49,5/1/17,2190,"Central City Liquor, Inc.",Des Moines,50314.0,"1460 2ND AVE\rDes Moines 50314\r(41.60566, -93...",POLK,Imported Flavored Vodka,DIAGEO AMERICAS,64573,...,12,750,$18.49,$27.74,NaN,$27.74,2.25,0.59,1,0


In [13]:
# obtain just the number of rows
liq[liq.bottles_sold.isnull()].shape[0]

66817

In [14]:
# divide this by the overall DataFrame to get a percent of missing values
liq[liq.bottles_sold.isnull()].shape[0] / liq.shape[0]

0.15614257705241363

Let's do the same for `sale`.

In [15]:
liq[liq.sale.isnull()].shape[0] / liq.shape[0]

0.06775518025439156

Collectively, we are missing about 16% of data on the number of bottles sold in a given daily transaction, and about 7% of the data on total sale value for a given number of items in a single day.

### Filling in missing data

How we fill in data depends largely on why it is missing (types of missingness) and what sampling we have available to us.

We may:

- Delete missing data altogether
- Fill in missing data with:
    - The average of the column
    - The median of the column
    - A predicted amount based on other factors
- Collect more data:
    - Resample the population
    - Followup with the authority providing data that is missing


In our case, let's focus on handling `bottles_sold`.

In [16]:
# Can we identify a pattern of missingness (no)
liq[liq.bottles_sold.isnull()]

,date,store_number,store_name,city,zip_code,location,county,category_name,vendor_name,item_number,...,pack,bottle_vol_ml,state_bottle_cost,state_bottle_retail,bottles_sold,sale,volumne_sold_l,volume_sold_gal,is_may_2017,is_may_2018
5,5/3/17,5105,Three Brothers Liquors,North Liberty,52317.0,585 HIGHWAY 965\rNorth Liberty 52317\r(41.7381...,JOHNSON,American Vodka,A V BRANDS INC,937040,...,6,750,$21.99,$32.99,NaN,NaN,1.50,0.39,1,0
7,5/1/17,4301,Sahota Food Mart,Des Moines,50320.0,"1805 SE 14th St\rDes Moines 50320\r(41.57222, ...",POLK,American Vodkas,Laird & Company,35914,...,24,375,$1.93,$2.90,NaN,$69.60,0.37,0.09,1,0
10,5/1/17,4301,Sahota Food Mart,Des Moines,50320.0,"1805 SE 14th St\rDes Moines 50320\r(41.57222, ...",POLK,Imported Brandies,Jim Beam Brands,47790,...,24,200,$4.72,$7.08,NaN,$169.92,0.40,0.10,1,0
16,5/1/17,4301,Sahota Food Mart,Des Moines,50320.0,"1805 SE 14th St\rDes Moines 50320\r(41.57222, ...",POLK,100% Agave Tequila,DIAGEO AMERICAS,89175,...,6,750,$32.99,$49.49,NaN,$49.49,2.25,0.59,1,0
30,5/1/17,4617,Lickety Liquor,Des Moines,50317.0,2501 HUBBELL AVE\rDes Moines 50317\r(41.607866...,POLK,Tennessee Whiskies,Brown Forman Corp.,86670,...,12,750,$15.57,$23.36,NaN,$23.36,1.50,0.39,1,0
39,5/1/17,4617,Lickety Liquor,Des Moines,50317.0,2501 HUBBELL AVE\rDes Moines 50317\r(41.607866...,POLK,Imported Cordials & Liqueurs,"Sidney Frank Importing Company, Inc.",65253,...,48,200,$3.51,$5.27,NaN,$252.96,0.20,0.05,1,0
40,5/1/17,4617,Lickety Liquor,Des Moines,50317.0,2501 HUBBELL AVE\rDes Moines 50317\r(41.607866...,POLK,100% Agave Tequila,Patron Spirits Company,88294,...,12,375,$14.50,$21.75,NaN,$261.00,0.37,0.09,1,0
42,5/1/17,4617,Lickety Liquor,Des Moines,50317.0,2501 HUBBELL AVE\rDes Moines 50317\r(41.607866...,POLK,Canadian Whiskies,SAZERAC NORTH AMERICA,12407,...,12,"1,000",$5.50,$8.25,NaN,$8.25,3.00,0.79,1,0
44,5/1/17,2190,"Central City Liquor, Inc.",Des Moines,50314.0,"1460 2ND AVE\rDes Moines 50314\r(41.60566, -93...",POLK,American Vodkas,McCormick Distilling Co.,36908,...,6,"1,750",$7.47,$11.21,NaN,$67.26,1.75,0.46,1,0
49,5/1/17,2190,"Central City Liquor, Inc.",Des Moines,50314.0,"1460 2ND AVE\rDes Moines 50314\r(41.60566, -93...",POLK,Imported Flavored Vodka,DIAGEO AMERICAS,64573,...,12,750,$18.49,$27.74,NaN,$27.74,2.25,0.59,1,0


In [17]:
# Do the missing values have a significantly different five number summary than non-missing?
liq[liq.bottles_sold.isnull()].describe()

,store_number,zip_code,item_number,pack,bottles_sold,volumne_sold_l,volume_sold_gal,is_may_2017,is_may_2018
count,66817.000000,66817.000000,66817.000000,66817.000000,0.0,66817.000000,66817.000000,66817.000000,66817.000000
mean,3791.062170,51280.016149,46513.278821,12.391263,NaN,1.956703,0.511402,0.489905,0.510095
std,1091.560203,986.335237,53291.422537,7.799176,NaN,5.707728,1.508028,0.499902,0.499902
min,2106.000000,50002.000000,139.000000,1.000000,NaN,0.020000,0.000000,0.000000,0.000000
25%,2616.000000,50317.000000,27175.000000,6.000000,NaN,0.750000,0.190000,0.000000,0.000000
50%,3857.000000,51105.000000,38177.000000,12.000000,NaN,1.500000,0.390000,0.000000,1.000000
75%,4806.000000,52314.000000,64858.000000,12.000000,NaN,2.000000,0.520000,1.000000,1.000000
max,9925.000000,56201.000000,995604.000000,48.000000,NaN,350.000000,92.460000,1.000000,1.000000


In [18]:
# full dataset 5-number summary
liq.describe()

,store_number,zip_code,item_number,pack,bottles_sold,volumne_sold_l,volume_sold_gal,is_may_2017,is_may_2018
count,427923.000000,427909.000000,427923.000000,427923.000000,361106.000000,427923.000000,427923.000000,427923.000000,427923.000000
mean,3789.924475,51275.466083,46367.248790,12.423403,2.254344,1.984745,0.518811,0.489308,0.510692
std,1090.596072,986.858053,52774.045141,7.849579,4.021036,6.175569,1.631609,0.499886,0.499886
min,2106.000000,50002.000000,139.000000,1.000000,0.000000,0.010000,0.000000,0.000000,0.000000
25%,2616.000000,50317.000000,27125.000000,6.000000,1.000000,0.750000,0.190000,0.000000,0.000000
50%,3849.000000,51104.000000,38177.000000,12.000000,1.000000,1.500000,0.390000,0.000000,1.000000
75%,4802.000000,52314.000000,64762.000000,12.000000,3.000000,2.000000,0.520000,1.000000,1.000000
max,9937.000000,56201.000000,998546.000000,48.000000,315.000000,551.250000,145.620000,1.000000,1.000000


In [19]:
# check the difference between the two
liq.describe() - liq[liq.bottles_sold.isnull()].describe()

,store_number,zip_code,item_number,pack,bottles_sold,volumne_sold_l,volume_sold_gal,is_may_2017,is_may_2018
count,361106.000000,361092.000000,361106.000000,361106.000000,361106.0,361106.000000,361106.000000,361106.000000,361106.000000
mean,-1.137695,-4.550066,-146.030031,0.032141,NaN,0.028042,0.007408,-0.000598,0.000598
std,-0.964132,0.522816,-517.377396,0.050403,NaN,0.467841,0.123581,-0.000016,-0.000016
min,0.000000,0.000000,0.000000,0.000000,NaN,-0.010000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,-50.000000,0.000000,NaN,0.000000,0.000000,0.000000,0.000000
50%,-8.000000,-1.000000,0.000000,0.000000,NaN,0.000000,0.000000,0.000000,0.000000
75%,-4.000000,0.000000,-96.000000,0.000000,NaN,0.000000,0.000000,0.000000,0.000000
max,12.000000,0.000000,2942.000000,0.000000,NaN,201.250000,53.160000,0.000000,0.000000


It appears the two samples do not have *significant* differences! (We could run statistical tests, but...another day.)

Now, this makes sense! We did randomly drop values, afterall.

Option 1: Drop the missing values.

In [20]:
# drops rows where any row has a missing value - this does not happen *in place*, so we are not actually dropping
liq.dropna()

,date,store_number,store_name,city,zip_code,location,county,category_name,vendor_name,item_number,...,pack,bottle_vol_ml,state_bottle_cost,state_bottle_retail,bottles_sold,sale,volumne_sold_l,volume_sold_gal,is_may_2017,is_may_2018
0,5/2/17,5286,Sauce,Iowa City,52240.0,"108, College\rIowa City 52240\r",JOHNSON,Blended Whiskies,Laird & Company,23827,...,12,"1,000",$4.40,$6.60,1.0,$79.20,1.00,0.26,1,0
1,5/1/17,4169,Super Quick 2 / Hubbell,Des Moines,50317.0,1824 Hubbell Ave\rDes Moines 50317\r,POLK,Canadian Whiskies,CONSTELLATION BRANDS INC,11773,...,48,200,$1.56,$2.34,1.0,$112.32,0.20,0.05,1,0
2,5/1/17,2641,Hy-Vee Drugstore / Council Bluffs,Council Bluffs,51501.0,757 W Broadway\rCouncil Bluffs 51501\r(41.2616...,POTTAWATTA,American Cordials & Liqueur,SAZERAC NORTH AMERICA,84207,...,10,600,$6.00,$9.00,1.0,$9.00,0.60,0.15,1,0
3,5/1/17,2641,Hy-Vee Drugstore / Council Bluffs,Council Bluffs,51501.0,757 W Broadway\rCouncil Bluffs 51501\r(41.2616...,POTTAWATTA,American Cordials & Liqueur,SAZERAC NORTH AMERICA,84197,...,10,600,$6.00,$9.00,1.0,$9.00,0.60,0.15,1,0
4,5/3/17,2565,Hy-Vee Food Store / Spencer,Spencer,51301.0,"819 N Grand Ave\rSpencer 51301\r(43.145897, -9...",CLAY,Mixto Tequila,LUXCO INC,89448,...,6,"1,750",$12.00,$18.00,3.0,$18.00,5.25,1.38,1,0
6,5/1/17,4301,Sahota Food Mart,Des Moines,50320.0,"1805 SE 14th St\rDes Moines 50320\r(41.57222, ...",POLK,American Brandies,CONSTELLATION BRANDS INC,53214,...,24,375,$3.22,$4.83,1.0,$115.92,0.37,0.09,1,0
8,5/1/17,4301,Sahota Food Mart,Des Moines,50320.0,"1805 SE 14th St\rDes Moines 50320\r(41.57222, ...",POLK,Imported Vodkas,BACARDI USA INC,34359,...,12,200,$5.00,$7.50,1.0,$90.00,0.20,0.05,1,0
9,5/1/17,4301,Sahota Food Mart,Des Moines,50320.0,"1805 SE 14th St\rDes Moines 50320\r(41.57222, ...",POLK,Imported Vodkas,BACARDI USA INC,34423,...,12,375,$9.00,$13.50,1.0,$162.00,0.37,0.09,1,0
11,5/1/17,4301,Sahota Food Mart,Des Moines,50320.0,"1805 SE 14th St\rDes Moines 50320\r(41.57222, ...",POLK,Imported Brandies,REMY COINTREAU USA INC,49084,...,12,375,$12.90,$19.35,1.0,$232.20,0.37,0.09,1,0
12,5/1/17,4301,Sahota Food Mart,Des Moines,50320.0,"1805 SE 14th St\rDes Moines 50320\r(41.57222, ...",POLK,Whiskey Liqueur,SAZERAC COMPANY INC,64868,...,6,"1,750",$15.33,$23.00,1.0,$138.00,1.75,0.46,1,0


Option 2: Fill in missing values

Traditionally, we fill missing data with a median, average, or modelled value. Let's see the five-number-summary of the column of interest to decide.

In [21]:
liq.bottles_sold.describe()

count    361106.000000
mean          2.254344
std           4.021036
min           0.000000
25%           1.000000
50%           1.000000
75%           3.000000
max         315.000000
Name: bottles_sold, dtype: float64

In this given case, we may opt to fill our data in with the *median* (50%) rather than the *mean* because we see such a positive skew. The most commonly processed transaction is on bottles that are single order.

In [22]:
# get the 50th percentile
liq.bottles_sold.quantile()

1.0

In [23]:
# fill in missing data with 50th percentile -- note we *are* making this change in place
liq.bottles_sold.fillna(value=liq.bottles_sold.quantile(), inplace = True)

In [24]:
# check total number of missing values
liq.isnull().sum()

date                       0
store_number               0
store_name                 0
city                      14
zip_code                  14
location                  14
county                    14
category_name            421
vendor_name                0
item_number                0
item_description           0
pack                       0
bottle_vol_ml              0
state_bottle_cost          0
state_bottle_retail        0
bottles_sold               0
sale                   28994
volumne_sold_l             0
volume_sold_gal            0
is_may_2017                0
is_may_2018                0
dtype: int64

They're gone!

Now, to be fair, we may want to investigate our missing values *even more*! What if counties with larger orders, on balance, are more likely to be missing from our dataset? This would skew our data unfairly.

Even determining how to fill in missing data requires careful exploratory data analysis!

## Groupby Statements

In Pandas, groupby statements are similar to pivot tables in that they allow us to segment our population to a specific subset.

For example, if we want to know the average number of bottles sold and pack sizes per city, a groupby statement would make this task much more straightforward.


To think how a groupby statement works, think about it like this:

- **Split:** Separate our DataFrame by a specific attribute
- **Apply:** Determine how categories are going to be mathematically incorporated. For example, if there are multiple store locations in one city, do we want the average amount across all stores, the total amount for the stores, or perhaps even the highest amount for a single store per city?
- **Combine:** Put our DataFrame back together.

![](http://i.imgur.com/yjNkiwL.png)

Let's try it out!

In [25]:
# groupby city - take the average for each column when combining back together
liq.groupby('city').mean()

,store_number,zip_code,item_number,pack,bottles_sold,volumne_sold_l,volume_sold_gal,is_may_2017,is_may_2018
city,,,,,,,,,
Ackley,4481.150000,50601.000000,37727.300000,11.962500,1.881250,1.797938,0.470188,0.450000,0.550000
Adair,4519.260870,50002.000000,36824.826087,13.637681,3.028986,2.199275,0.574348,0.333333,0.666667
Adel,4596.451939,50003.000000,41947.055649,11.873524,2.357504,2.387352,0.625110,0.522766,0.477234
Afton,4531.000000,50830.000000,41484.700000,9.833333,1.150000,1.757167,0.458833,0.466667,0.533333
Akron,4993.326923,51001.000000,40348.153846,14.692308,3.548077,2.770865,0.726442,0.528846,0.471154
Albert City,5266.000000,50510.000000,41968.312500,18.000000,1.375000,0.926250,0.238750,0.312500,0.687500
Albia,3423.772636,52531.000000,46534.146881,11.374245,2.448692,3.018732,0.792334,0.569416,0.430584
Albion,5378.000000,50005.000000,38616.054054,12.702703,1.324324,1.487568,0.386486,0.783784,0.216216
Alburnett,5269.000000,52202.000000,50115.125000,15.541667,1.791667,1.439583,0.374167,0.000000,1.000000


In [26]:
# perhaps we want *just* bottles sold from the above
liq.groupby('city').bottles_sold.mean()

city
Ackley             1.881250
Adair              3.028986
Adel               2.357504
Afton              1.150000
Akron              3.548077
Albert City        1.375000
Albia              2.448692
Albion             1.324324
Alburnett          1.791667
Alden              2.331492
Algona             1.823031
Allison            2.598485
Alta               1.407407
Altoona            1.950881
Amana              2.435897
Ames               2.081587
Anamosa            1.848784
Anita              2.130435
Ankeny             2.360507
Anthon             1.153846
Arlington          2.213115
Armstrong          1.900000
Arnold's Park      1.891509
Arnolds Park       2.010698
Atkins             1.747863
Atlantic           2.148601
Audubon            2.845912
Aurelia            1.176471
Avoca              1.981439
Baldwin            1.236842
                     ...   
Wapello            3.131991
Washburn           2.281250
Washington         1.887414
Washta             2.200000
Waterloo       

In [27]:
# or maybe, we want the biggest single transaction per county
liq.groupby('city').bottles_sold.max()

city
Ackley               6.0
Adair               12.0
Adel                50.0
Afton                3.0
Akron               12.0
Albert City          4.0
Albia              197.0
Albion               3.0
Alburnett            4.0
Alden                6.0
Algona              50.0
Allison             24.0
Alta                 5.0
Altoona            120.0
Amana               12.0
Ames               200.0
Anamosa             50.0
Anita               12.0
Ankeny             156.0
Anthon               4.0
Arlington            6.0
Armstrong            4.0
Arnold's Park        8.0
Arnolds Park         8.0
Atkins              10.0
Atlantic           100.0
Audubon             24.0
Aurelia              2.0
Avoca                8.0
Baldwin              3.0
                   ...  
Wapello             12.0
Washburn            10.0
Washington         200.0
Washta               6.0
Waterloo           128.0
Waukee              75.0
Waukon             100.0
Waverly            100.0
Webster City       1

In [28]:
# in fact, we can 'apply' a mean and max at once- plus count and min!
liq.groupby('city').bottles_sold.agg(['count', 'mean', 'min', 'max'])

,count,mean,min,max
city,,,,
Ackley,160,1.881250,1.0,6.0
Adair,69,3.028986,1.0,12.0
Adel,593,2.357504,1.0,50.0
Afton,60,1.150000,1.0,3.0
Akron,104,3.548077,1.0,12.0
Albert City,16,1.375000,1.0,4.0
Albia,497,2.448692,1.0,197.0
Albion,37,1.324324,1.0,3.0
Alburnett,24,1.791667,1.0,4.0


In [29]:
# sort by largest average; grab top 10 cities in Iowa by average liquor store bottle size purchase
liq.groupby('city').bottles_sold.agg(['count', 'mean', 'min', 'max']).sort_values(by='mean', ascending=False).head(10)

,count,mean,min,max
city,,,,
Delmar,29,5.586207,1.0,24.0
Grand Junction,45,5.355556,1.0,20.0
Donnellson,50,5.100000,1.0,12.0
Earling,4,5.000000,3.0,7.0
Lovilia,21,4.809524,1.0,12.0
Winfield,20,4.750000,1.0,18.0
Dakota City,33,4.606061,1.0,12.0
Early,38,4.263158,1.0,12.0
Exira,43,4.046512,1.0,12.0


In [30]:
# groupby creates a groupby object - it needs to be told how to aggregate things together
liq.groupby('city').bottles_sold

In [31]:
liq.groupby('city').bottles_sold.count()

city
Ackley               160
Adair                 69
Adel                 593
Afton                 60
Akron                104
Albert City           16
Albia                497
Albion                37
Alburnett             24
Alden                181
Algona              1181
Allison              132
Alta                  27
Altoona             3746
Amana                 39
Ames               11068
Anamosa             1521
Anita                184
Ankeny              8047
Anthon               104
Arlington             61
Armstrong             30
Arnold's Park        636
Arnolds Park        1776
Atkins               234
Atlantic            1716
Audubon              318
Aurelia               17
Avoca                431
Baldwin               38
                   ...  
Wapello              447
Washburn              32
Washington          2034
Washta                10
Waterloo           14255
Waukee              2697
Waukon              1017
Waverly             2111
Webster City        

In [32]:
# top 10 cities by counts of active liquor stores
liq.groupby('city').bottles_sold.count().sort_values(ascending=False).head(10)

city
Des Moines         34753
Cedar Rapids       30651
Davenport          17709
Waterloo           14255
Sioux City         13696
Dubuque            12986
Iowa City          11962
Council Bluffs     11641
West Des Moines    11077
Ames               11068
Name: bottles_sold, dtype: int64

## Apply functions for column operations

Apply functions allow us to perform a complex operation across an entire columns highly efficiently.

For example, recall our `sale` data is formatted in an unhelpful way (strings, not floats):                                                      

In [33]:
liq.dtypes

date                    object
store_number             int64
store_name              object
city                    object
zip_code               float64
location                object
county                  object
category_name           object
vendor_name             object
item_number              int64
item_description        object
pack                     int64
bottle_vol_ml           object
state_bottle_cost       object
state_bottle_retail     object
bottles_sold           float64
sale                    object
volumne_sold_l         float64
volume_sold_gal        float64
is_may_2017              int64
is_may_2018              int64
dtype: object

In [34]:
# first sale value
liq.sale[0]

'$79.20 '

We need to convert this value to a float, and without the dollar sign.

**Apply functions** allow us to write a function that cleans a single value, and then we *apply* that function to a whole column. (It's like a for loop, but way more efficient as an operation!)

Writing them follows a familiar three steps:

1. Write a function that creates the desired output on a single value
2. Test that function on one value of interest
3. Apply that function to the whole column

To start, let's write a function that converts an inputted value with a dollar sign to a float, and returns that float.

In [35]:
def dollars_to_float(value):
    
    # try to convert the inputted value to a float
    try:
        return float(value.strip('$'))
    
    # in the case of the value being a null value, we simply return a null
    except:
        return np.nan

Let's try our function on a value of interest or two.

In [36]:
liq.sale[0]

'$79.20 '

In [37]:
liq.sale[2]

'$9.00 '

In [38]:
dollars_to_float(liq.sale[0])

79.2

In [39]:
dollars_to_float(liq.sale[2])

9.0

Now, we apply this function to the whole column with the following syntax. Notice: we are going to create a new column (out of thin air!) called `sale_clean`.

In [40]:
liq['sale_clean'] = liq.sale.apply(dollars_to_float)

In [41]:
liq['sale_clean'][0:10]

0     79.20
1    112.32
2      9.00
3      9.00
4     18.00
5       NaN
6    115.92
7     69.60
8     90.00
9    162.00
Name: sale_clean, dtype: float64

Voila! Our first apply function.

**Your turn:** Identify one other column where we may want to write a new apply function, or use the one we just created for the purposes of cleaning up our dataset.

In [42]:
# identify a column to fix
liq.dtypes

date                    object
store_number             int64
store_name              object
city                    object
zip_code               float64
location                object
county                  object
category_name           object
vendor_name             object
item_number              int64
item_description        object
pack                     int64
bottle_vol_ml           object
state_bottle_cost       object
state_bottle_retail     object
bottles_sold           float64
sale                    object
volumne_sold_l         float64
volume_sold_gal        float64
is_may_2017              int64
is_may_2018              int64
sale_clean             float64
dtype: object

In [43]:
# sale bottle retail looks like it needs fixed
liq.state_bottle_retail [0]

'$6.60 '

In [44]:
# write a function to fix a single value in that columns
# I'll use the same function

In [45]:
dollars_to_float(liq.state_bottle_retail[0])

6.6

In [46]:
# apply that function across the whole column


In [47]:
liq['state_bottle_retail_clean'] = liq.state_bottle_retail.apply(dollars_to_float)

In [48]:
liq['state_bottle_retail_clean'][0:10]

0     6.60
1     2.34
2     9.00
3     9.00
4    18.00
5    32.99
6     4.83
7     2.90
8     7.50
9    13.50
Name: state_bottle_retail_clean, dtype: float64

## Wrap up

We've covered even more useful information! Here are the key takeaways:

- **Missing data** comes in many shapes and sizes. Before deciding how to handle it, we identify it exists. We then derive how the missingness is affecting our dataset, and make a determination about how to fill in values.

```python
# pro tip for identifying missing data
df.isnull().sum()
```

- **Grouby** statements are particularly useful for a subsection-of-interest analysis. Specifically, zooming in on one condition, and determining relevant statstics.

```python
# group by 
df.groupby('column').agg['count', 'mean', 'max', 'min']
```

- **Apply functions** help us clean values across an entire DataFrame column. They are *like* a for loop for cleaning, but many times more efficient. They follow a common pattern:
1. Write a function that works on a single value
2. Test that function on a single value
3. Apply that function to a whole column

(The most confusing part of apply functions is that we write them with *a single value* in mind, and then apply them to many single values at once.)